# Create DTA Branch

Creates a new DTA branch from a library major version.

**Operation:** `CREATE_BRANCH`

**What it does:**
- Copies all records from the specified library major version
- Creates new records with version tag: `{version}-{dta_id}-draft1`
- Sets `is_major_version=FALSE`, `is_dta_major=FALSE`, `is_current=TRUE`
- Registers the new draft in `md_version_registry`

**Parameters:**
- `dta_id` - DTA identifier (required)
- `library_type` - Library type (e.g., transfer_variables)
- `base_template_version` - Library version to branch from (e.g., "1.0")


In [ ]:
# Cell 1: Imports
import json
from clinical_data_standards_framework.versioning import (
    create_library_branch,
    get_version_registry_schema
)

print("="*80)
print("CREATE DTA BRANCH")
print("="*80)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
versioning_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="versioning"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog = globals_dict['catalog']
gold_schema = globals_dict['gold_schema']

# Versioning configuration from YAML
library_tables = versioning_dict.get('library_tables', [])
registry_table_name = versioning_dict.get('registry_table', 'md_version_registry')

print(f"Catalog: {catalog}")
print(f"Gold Schema: {gold_schema}")
print(f"Versioning Config: {len(library_tables)} library table(s)")
print(f"Created by: {created_by_principal}")


In [ ]:
# Cell 3: Get Widget Parameters (job-level parameters only)
dbutils.widgets.text("dta_id", "", "DTA ID")
dbutils.widgets.text("library_type", "", "Library Type (defaults from config)")
dbutils.widgets.text("base_template_version", "1.0", "Base Version")

dta_id = dbutils.widgets.get("dta_id")
library_type_param = dbutils.widgets.get("library_type")
base_template_version = dbutils.widgets.get("base_template_version")

# Default to first library type from config if not specified
if library_type_param:
    library_type = library_type_param
elif library_tables:
    library_type = library_tables[0].get('library_type', 'transfer_variables')
else:
    library_type = 'transfer_variables'

print(f"\nDTA ID: {dta_id}")
print(f"Library Type: {library_type} (from {'param' if library_type_param else 'config'})")
print(f"Base Version: {base_template_version}")

# Validate
if not dta_id:
    raise ValueError("dta_id parameter is required")


In [ ]:
# Cell 4: Configure Table Names (from config)
# Build library table map from config
library_table_map = {}
for lib_config in library_tables:
    lib_type = lib_config.get('library_type')
    lib_name = lib_config.get('name')
    lib_schema = lib_config.get('schema', gold_schema)
    if lib_type and lib_name:
        library_table_map[lib_type] = f"{catalog}.{lib_schema}.{lib_name}"

if library_type not in library_table_map:
    available = ", ".join(library_table_map.keys())
    raise ValueError(f"Unknown library_type: {library_type}. Available: {available}")

library_table = library_table_map[library_type]
registry_table = f"{catalog}.{gold_schema}.{registry_table_name}"

print(f"\nLibrary Table: {library_table}")
print(f"Registry Table: {registry_table}")


In [ ]:
# Cell 5: Ensure Registry Table Exists
if not spark.catalog.tableExists(registry_table):
    print(f"Creating version registry table: {registry_table}")
    empty_df = spark.createDataFrame([], schema=get_version_registry_schema())
    empty_df.write.format("delta").mode("overwrite").saveAsTable(registry_table)
    print(f"Created {registry_table}")
else:
    print(f"Registry table exists: {registry_table}")


In [ ]:
# Cell 6: Execute CREATE_BRANCH
print(f"\n{'='*80}")
print(f"Creating branch for DTA {dta_id} from version {base_template_version}...")
print(f"{'='*80}")

result_version = create_library_branch(
    spark=spark,
    library_table=library_table,
    registry_table=registry_table,
    dta_id=dta_id,
    base_template_version=base_template_version,
    library_type=library_type,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id
)

print(f"\nBranch created successfully!")
print(f"New version tag: {result_version}")


In [ ]:
# Cell 7: Set Task Values for Downstream
dbutils.jobs.taskValues.set(key="result_version", value=result_version)
dbutils.jobs.taskValues.set(key="dta_id", value=dta_id)
dbutils.jobs.taskValues.set(key="action", value="CREATE_BRANCH")

print(f"\nTask values set:")
print(f"  result_version: {result_version}")
print(f"  dta_id: {dta_id}")
print(f"  action: CREATE_BRANCH")
